#**Utilizamos el sentiment analysis por cluster para crear una nueva columna en los datos**




La idea es llevar información del entorno social hasta el dataset a nivel alumno

# Preparación de colab

## Importamos librerías

In [ ]:

import pandas as pd
import re
import matplotlib.pyplot as plt

import os
from glob import glob
import unicodedata # Preprocesado
import csv

import matplotlib.pyplot as plt
import seaborn as sns

## Conecto con drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path_dataset='drive/My Drive/Proyecto_BD_coders/Dataset/PISA_DROPOUT_RISK/'
path_rdo='drive/My Drive/keepcoding/Tweets/resultados/'
path_dataset_rdo='drive/My Drive/keepcoding/Tweets/dataset_with_tweets/'

# Importo dataset

In [ ]:
X_train = pd.read_csv(path_dataset+'X2_train.csv',index_col=0)
X_test = pd.read_csv(path_dataset+'X2_test.csv',index_col=0)
X_val = pd.read_csv(path_dataset+'X2_val.csv',index_col=0)


In [ ]:
print(f'Tengo {len(X_train)} filas en X_train')
print(f'Tengo {len(X_test)} filas en X_test')
print(f'Tengo {len(X_val)} filas en X_val')


Tengo 19953 filas en X_train
Tengo 6236 filas en X_test
Tengo 4989 filas en X_val


# Importo información de analisis de Twitter

In [ ]:
# aqui obtengo la clave de pregunta y cluster al que pertenece
cluster_preguntas=pd.read_csv(path_rdo+'Cluster_preguntas.csv')
cluster_preguntas=cluster_preguntas[['cluster','key']]
cluster_preguntas.head()

,cluster,key
0,0,ST011D17TA
1,0,ST011D18TA
2,0,ST011D19TA
3,0,ST197Q09HA
4,0,ST222Q03HA


In [ ]:
cluster_preguntas=pd.read_csv(path_rdo+'Cluster_preguntas.csv')

col_X=pd.DataFrame(X_train.columns,columns=['key'])
col_X=pd.merge(cluster_preguntas,col_X,on='key')


In [ ]:
# aqui obtengo los scores por cluster
cluster_score=pd.read_csv(path_rdo+'scores.csv')
cluster_score

,clusters,scores
0,cluster0,0.265285
1,cluster1,0.042614
2,cluster2,0.103554
3,cluster3,0.125381
4,cluster4,0.074955


# Creamos una función agrupe las respuestas del alumno según clusters

In [ ]:
def add_score_pond(X,cluster_preguntas,cluster_score):
  # me quedo con laspreguntas que han pasado la limpieza final para usar en el modelo y las relaciono con número de cluster
  col_X=pd.DataFrame(X.columns,columns=['key'])
  col_X=pd.merge(cluster_preguntas,col_X,on='key')
  for i in range(0,5):
    X['pond_cluster'+str(i)]=X.filter(col_X[col_X['cluster']==i]['key']).sum(axis=1)
    X['Score'+str(i)]=float(cluster_score[cluster_score['clusters']=='cluster'+str(i)]['scores'])
  # X['Score_pond']=(X['pond_cluster0']*X['Score0']+X['pond_cluster1']*X['Score1']+X['pond_cluster2']*X['Score2']+X['pond_cluster3']*X['Score3']+X['pond_cluster4']*X['Score4'])/(X['pond_cluster0']+X['pond_cluster1']+X['pond_cluster2']+X['pond_cluster3']+X['pond_cluster4'])
  X['Score_pond']=(X['pond_cluster0']*X['Score0']+X['pond_cluster1']*X['Score1']+X['pond_cluster2']*X['Score2']+X['pond_cluster3']*X['Score3']+X['pond_cluster4']*X['Score4'])
  X.drop(columns=['pond_cluster0','pond_cluster1','pond_cluster2','pond_cluster3','pond_cluster4','Score0','Score1','Score2','Score3','Score4'],inplace=True)
  return X



# Aplicamos función a X_train, test y validation

In [ ]:
X_train=add_score_pond(X_train,cluster_preguntas,cluster_score)
X_test=add_score_pond(X_test,cluster_preguntas,cluster_score)
X_val=add_score_pond(X_val,cluster_preguntas,cluster_score)


In [ ]:
X_train

,OCOD3,SENWT,CNTSCHID,BSMJ,ST001D01T,GCSELFEFF,PV8RTSN,PV10RCER,MASTGOAL,PV4RCER,...,PV1GLCM,PV7RCER,PV8RCER,PV9RTSN,PV10RTML,PV6RCER,PV5RCUN,PV10RCLI,PV7RTML,Score_pond
7575,5312,0.12762,72400642.0,24.98,9.0,-0.3942,454.112,521.570,-1.7101,482.296,...,515.221,432.307,439.821,417.673,418.863,501.880,493.764,532.562,500.678,1359.035613
168,2341,0.11360,72400299.0,76.49,10.0,-0.9245,555.810,621.970,-0.0940,573.922,...,581.228,671.652,605.917,604.273,528.853,617.100,612.626,595.217,567.502,1531.717462
17256,110,0.12794,72400127.0,60.92,8.0,-0.4038,610.075,513.757,1.8524,580.198,...,562.724,575.387,594.244,610.115,533.581,570.786,597.612,574.027,571.532,1401.820965
2052,5411,0.03512,72400661.0,46.38,9.0,-0.4720,355.136,332.079,-0.4347,394.915,...,366.160,362.860,352.015,302.830,360.158,362.000,330.098,280.031,348.090,1086.888366
9303,3411,0.42566,72400155.0,57.00,10.0,-0.1247,545.486,475.272,1.1969,662.077,...,616.622,526.312,605.057,503.340,525.220,485.852,613.356,519.077,555.490,1474.233579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19922,9704,0.04037,72400846.0,NaN,10.0,0.1904,534.190,526.735,-1.7101,480.259,...,523.184,527.459,530.419,551.293,516.259,555.971,487.542,509.415,520.106,1657.496072
20328,2211,0.11750,72400474.0,88.70,10.0,0.6656,422.288,388.590,1.1969,431.280,...,467.512,498.999,460.597,481.690,463.171,409.825,476.550,490.591,451.576,1219.652638
11990,2330,0.33835,72400130.0,82.41,10.0,0.8685,533.491,621.016,0.8347,542.699,...,741.616,602.953,646.030,588.149,612.389,619.344,594.347,508.245,599.624,1530.573952
23839,3431,0.12353,72400345.0,50.15,9.0,-0.9477,319.283,296.859,-1.3162,344.289,...,434.825,298.335,369.732,273.945,280.618,390.240,331.695,345.989,334.437,988.799521


# Guardamos dataset con columna de twitter añadida

In [ ]:
# X_train.to_csv(path_dataset_rdo+'X_train_scoretwitter.csv')
# X_test.to_csv(path_dataset_rdo+'X_test_scoretwitter.csv')
# X_val.to_csv(path_dataset_rdo+'X_val_scoretwitter.csv')

In [ ]:
X_train.to_csv(path_dataset_rdo+'X_train_scoretwitter_sinpond.csv')
X_test.to_csv(path_dataset_rdo+'X_test_scoretwitter_sinpond.csv')
X_val.to_csv(path_dataset_rdo+'X_val_scoretwitter_sinpond.csv')

In [ ]:
X_train.columns

Index(['OCOD3', 'SENWT', 'CNTSCHID', 'BSMJ', 'ST001D01T', 'GCSELFEFF',
       'PV8RTSN', 'PV10RCER', 'MASTGOAL', 'PV4RCER', 'GCAWARE', 'PV4RTML',
       'PV10RTSN', 'PV4RCLI', 'PV1RCER', 'PV7RCUN', 'PV1RTML', 'PV3RCUN',
       'PV5RTSN', 'PV6RTSN', 'PV8RTML', 'PV3RCER', 'PV9RCLI', 'PV8RCLI',
       'PV9MATH', 'PV6RTML', 'PV9READ', 'PV8RCUN', 'PV3READ', 'PV3RTSN',
       'PV2RTSN', 'PV1GLCM', 'PV7RCER', 'PV8RCER', 'PV9RTSN', 'PV10RTML',
       'PV6RCER', 'PV5RCUN', 'PV10RCLI', 'PV7RTML', 'Score_pond'],
      dtype='object')